In [17]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [18]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
file_path = Path("Resources/lending_data.csv")
df = pd.read_csv(file_path)
# Review the DataFrame
df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [19]:
# Separate the data into labels and features

# Separate the y variable, the labels

y = df["loan_status"].values.reshape(-1,1)

# Separate the X variable, the features
X = df.copy()
X.drop("loan_status", axis=1,inplace=True)

In [20]:
# Review the y variable Series
y[:5]

array([[0],
       [0],
       [0],
       [0],
       [0]], dtype=int64)

In [21]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [24]:
# Check the balance of our target values
values, counts = np.unique(y, return_counts=True)

dict(zip(values, counts))

{0: 75036, 1: 2500}

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [27]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [28]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using training data
classifier.fit(X_train, y_train)

C:\Users\src54\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [44]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [45]:
# Print the balanced_accuracy score of the model
bas = balanced_accuracy_score(y_test, predictions)
bas

0.9936781215845847

In [32]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, predictions)

array([[18663,   102],
       [   56,   563]], dtype=int64)

In [34]:
# Print the classification report for the model
target_names = ["Healthy Loan", "High-Risk Loan"]
print(classification_report(y_test, predictions, target_names=target_names))

                precision    recall  f1-score   support

  Healthy Loan       1.00      0.99      1.00     18765
High-Risk Loan       0.85      0.91      0.88       619

      accuracy                           0.99     19384
     macro avg       0.92      0.95      0.94     19384
  weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic model was nearly perfect in predicting healthy loans, with a rounded f1-score of 1. High risk loans proved to be a more complicated matter, with a f1-score of only .88 due to both false negatives and positives. It cannot be relied upon in this regard except as a supplement to other testing.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [39]:
#This portion of code is sampled from https://imbalanced-learn.org/stable/over_sampling.html
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ros = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled = ros.fit_resample(X, y)

In [40]:
# Count the distinct values of the resampled labels data
values, counts = np.unique(y_resampled, return_counts=True)

dict(zip(values, counts))

{0: 75036, 1: 75036}

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [50]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model

#Already initialized previously

# Fit the model using the resampled training data
classifier.fit(X_resampled, y_resampled)

# Make a prediction using the testing data
predictions = classifier.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [52]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_test, predictions)

0.9936781215845847

In [53]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, predictions)

array([[18649,   116],
       [    4,   615]], dtype=int64)

In [54]:
# Print the classification report for the model
target_names = ["Healthy Loan", "High-Risk Loan"]
print(classification_report(y_test, predictions, target_names=target_names))

                precision    recall  f1-score   support

  Healthy Loan       1.00      0.99      1.00     18765
High-Risk Loan       0.84      0.99      0.91       619

      accuracy                           0.99     19384
     macro avg       0.92      0.99      0.95     19384
  weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** Once again, the logistic regression model did quite well for predicting healthy loans, with a high rate of precision and recall. However, the model using oversampled data also showed a marked improvement in recall over the initial sample, with a score of .99. This means it is quite effective at identifying high-risk loans with very few false positives. However, it is still marking a substantial number of high-risk loans as healthy, meaning that once again this model should only be used a supplemental option to further analysis.